In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
plotly.tools.set_credentials_file(username='lauraraynes', api_key='Bm8XEYfXMQzXyHxVptSS')
import plotly.plotly as py
import plotly.figure_factory as ff

data = pd.read_csv("real_estate_db.csv", encoding='ISO-8859-1')
all_fips = pd.read_csv("all-geocodes-v2016.csv", encoding='ISO-8859-1')

In [17]:
rent_40_df = data[["rent_gt_40", 'STATEID', "COUNTYID", "state"]]
rent_40_df.head()

rent_gt_40  STATEID  COUNTYID   state
0     0.15135        2        16  Alaska
1     0.20455        2        20  Alaska
2     0.54368        2        20  Alaska
3     0.27286        2        20  Alaska
4     0.24829        2        20  Alaska

In [18]:
rent_40_df.loc[:,"STATEID"] = rent_40_df["STATEID"].map("{:02d}".format)
rent_40_df.loc[:,"COUNTYID"] = rent_40_df["COUNTYID"].map("{:003d}".format)

In [19]:
rent_40_df.head()

rent_gt_40 STATEID COUNTYID   state
0     0.15135      02      016  Alaska
1     0.20455      02      020  Alaska
2     0.54368      02      020  Alaska
3     0.27286      02      020  Alaska
4     0.24829      02      020  Alaska

In [20]:
rent_40_df["FIPS"] = rent_40_df["STATEID"].map(str) + rent_40_df["COUNTYID"].map(str)

In [21]:
rent_40_clean = rent_40_df.dropna(how='any')
rent_40_clean.head()

rent_gt_40 STATEID COUNTYID   state   FIPS
0     0.15135      02      016  Alaska  02016
1     0.20455      02      020  Alaska  02020
2     0.54368      02      020  Alaska  02020
3     0.27286      02      020  Alaska  02020
4     0.24829      02      020  Alaska  02020

In [35]:
grouped_data = rent_40_clean.groupby(["FIPS"])
mean_rent_40 = grouped_data.mean().reset_index()
print("Maximum: " + str(mean_rent_40["rent_gt_40"].max()))
print("Median: " + str(mean_rent_40["rent_gt_40"].median()))
mean_rent_40.head()

Maximum: 0.86364
Median: 0.3095330769230769


FIPS  rent_gt_40
0  01001    0.348900
1  01003    0.312674
2  01005    0.386574
3  01007    0.254480
4  01009    0.361050

In [26]:
#Add leading zeros
all_fips.loc[:,"State Code (FIPS)"] = all_fips["State Code (FIPS)"].map("{:02d}".format)
all_fips.loc[:,"County Code (FIPS)"] = all_fips["County Code (FIPS)"].map("{:003d}".format)

In [36]:
#Add column for full FIPS
all_fips["FIPS"] = all_fips["State Code (FIPS)"].map(str) + all_fips["County Code (FIPS)"].map(str)

#Keep only counties
county_fips = all_fips[all_fips["County Code (FIPS)"] != "000"].reset_index()

clean_fips = county_fips[["Area Name (including legal/statistical area description)", "FIPS"]]
clean_fips.head()

Area Name (including legal/statistical area description)   FIPS
0                                     Autauga County        01001
1                                     Baldwin County        01003
2                                     Barbour County        01005
3                                        Bibb County        01007
4                                      Blount County        01009

In [40]:
merge_table = pd.merge(clean_fips, mean_rent_40, how="outer")

#Fill missing counties with median
merge_table["rent_gt_40"].fillna(0.30953, inplace=True)

merge_table.head()

Area Name (including legal/statistical area description)   FIPS  rent_gt_40
0                                     Autauga County        01001    0.348900
1                                     Baldwin County        01003    0.312674
2                                     Barbour County        01005    0.386574
3                                        Bibb County        01007    0.254480
4                                      Blount County        01009    0.361050

In [41]:
colorscale = ["00E53D","00E219","09DF00","2BDD00","4DDA00","6DD800","8DD500","ACD300",\
              "CBD000","CEB300","CB9200","C97100","C65000","C43100","C11200","BF000A"]    
    
endpts = list(np.linspace(0, 0.86364, len(colorscale) - 1))
fips = merge_table["FIPS"]
values = merge_table["rent_gt_40"]

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['usa'],
    binning_endpoints=endpts, colorscale=colorscale,
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 0},
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
    asp=2.9, title='Percentage of Population whose Rent Exceeds 40% of Income',
    legend_title='Percentage of Population')

py.iplot(fig, filename='choropleth_full_usa')

C:\Users\Laura\Anaconda3.1\lib\site-packages\plotly\plotly\plotly.py:224: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



The draw time for this plot will be slow for all clients.


C:\Users\Laura\Anaconda3.1\lib\site-packages\plotly\api\v1\clientresp.py:40: UserWarning:

Estimated Draw Time Too Long



In [49]:
hundred = mean_rent_40.sort_values(["rent_gt_40"], ascending = False).head(100)
top_100 = hundred.reset_index(drop = True)
top_100.head()

FIPS  rent_gt_40
0  26003     0.86364
1  51007     0.82320
2  72019     0.79675
3  51750     0.78646
4  28111     0.77647

In [50]:
northeast = []
midwest = []
south = []
west = []
territory = []

index = 0

for value in top_100["FIPS"]:
    if value[:2] in ("09", "23", "25", "33", "44", "50", "34", "36", "42"):
        northeast.append(top_100.loc[index, "rent_gt_40"])
    elif value[:2] in ("17", "18", "26", "39", "55", "19", "20", "27", "29", "31", "38", "46"):
        midwest.append(top_100.loc[index, "rent_gt_40"])
    elif value[:2] in ("10", "11", "12", "13", "24", "37", "45", "51", "54", "01", "21", "28", "47", "05", "22", "40", "48"):
        south.append(top_100.loc[index, "rent_gt_40"])
    elif value[:2] in ("08", "16", "30", "32", "35", "49", "56", "02", "06", "15", "41", "53"):
        west.append(top_100.loc[index, "rent_gt_40"])
    else:
        territory.append(top_100.loc[index, "rent_gt_40"])   

    index += 1
    
print(len(northeast))
print(len(midwest))
print(len(south))
print(len(west))
print(len(territory))



1
10
63
10
16
